In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

import numpy as np

from collections import OrderedDict, defaultdict

In [67]:
class ClientDataset(Dataset):
    def __init__(self, img_tensors, lbl_tensors, transform=None):
        self.img_tensors = img_tensors
        self.lbl_tensors = lbl_tensors
        self.transform = transform
    
    def __len__(self):
        return self.lbl_tensors.shape[0]
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        return self.img_tensors[idx], self.lbl_tensors[idx]
    
def create_client_data_loaders(client_nums, data_folder, batch_size, random_mode=False):
    data_loaders = []
    for idx in range(client_nums):
        # loading data to tensors
        img_tensor_file = data_folder + f'client_{idx}_img.pt'
        lbl_tensor_file = data_folder + f'client_{idx}_lbl.pt'
        img_tensors = torch.load(img_tensor_file) # this contains 494 images, currently 76
        lbl_tensors = torch.load(lbl_tensor_file)

        # creating a dataset which can be fed to dataloader
        client_dataset = ClientDataset(img_tensors, lbl_tensors)
        data_loader = DataLoader(client_dataset, batch_size=batch_size, shuffle=random_mode)
        data_loaders.append(data_loader)
    return data_loaders

In [78]:
# Save the tensor of images and labels for clients
username = 'fnx11'
data_folder = f'/home/{username}/thesis/codes/Playground/data/fed_data/'
client_nums = 20
client_data_loaders = create_client_data_loaders(client_nums, data_folder, 8)

### Let's see some of the loaders

In [79]:
import matplotlib.pyplot as plt
%matplotlib inline

# helper function to un-normalize and display an image
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    plt.imshow(np.transpose(img, (1, 2, 0)))

def visualize(images, labels):
    images = images.numpy() # convert images to numpy for display
    # plot the images in the batch, along with the corresponding labels
    fig = plt.figure(figsize=(25, 4))
    # display 20 images
    for idx in np.arange(8):
        ax = fig.add_subplot(2, 8/2, idx+1, xticks=[], yticks=[])
        imshow(images[idx])
        ax.set_title(classes[labels[idx]])
    

In [84]:
client_iter = iter(client_data_loaders[15])
# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']
for step in range(len(client_iter)):
    images, labels = next(client_iter)  
#     visualize(images, labels)
    print(images.shape)
    print(labels.shape)
    break

torch.Size([8, 3, 32, 32])
torch.Size([8])


In [1]:
from sklearn.cluster import KMeans
import numpy as np

In [6]:
arr1 = np.array([1,2,3,4,5,80,90,100,110,120]).reshape(-1, 1)
arr2 = np.array([11,12,13,14,15,16,17,20,220,230]).reshape(-1, 1)
arr3 = np.array([100,120,105,8,9,10,11,12,13,14]).reshape(-1, 1)


In [7]:
kmeans1 = KMeans(n_clusters=2, random_state=0).fit(arr1)
centers1 = kmeans1.cluster_centers_
print(centers1)

kmeans2 = KMeans(n_clusters=2, random_state=0).fit(arr2)
centers2 = kmeans2.cluster_centers_
print(centers2)

kmeans3 = KMeans(n_clusters=2, random_state=0).fit(arr3)
centers3 = kmeans3.cluster_centers_
print(centers3)

[[100.]
 [  3.]]
[[ 14.75]
 [225.  ]]
[[ 11.        ]
 [108.33333333]]


In [8]:
mins = []

labels1 = kmeans1.predict(arr1)
val1, count1 = np.unique(labels1, return_counts=True)
print(count1)
mins.append(np.min(count1))

labels2 = kmeans2.predict(arr2)
val2, count2 = np.unique(labels2, return_counts=True)
print(count2)
mins.append(np.min(count2))

labels3 = kmeans3.predict(arr3)
val3, count3 = np.unique(labels3, return_counts=True)
print(count3)
mins.append(np.min(count3))

print(mins)

[5 5]
[8 2]
[7 3]
[5, 2, 3]


In [11]:
min_arr = np.array(mins).reshape(-1, 1)
kmeans = KMeans(n_clusters=2, random_state=0).fit(min_arr)
centers = kmeans.cluster_centers_
print(centers)

labels = kmeans.predict(min_arr)
print(labels)
val, count = np.unique(labels, return_counts=True)

[[5. ]
 [2.5]]
[0 1 1]


In [4]:
import numpy as np

In [5]:
arr = np.array([4,2,1,3,6,5])
brr = np.argsort(arr)[-3:]

In [6]:
print(arr)
print(brr)

[4 2 1 3 6 5]
[0 5 4]


In [10]:
arr[brr]

array([4, 5, 6])

In [11]:
len(arr)

6

In [12]:
arr.size

6

In [16]:
from sklearn.metrics.pairwise import cosine_similarity
arr1 = np.arange(10).reshape(1, -1)
arr2 = np.arange(10).reshape(1, -1)

print(cosine_similarity(arr1, arr2)[0][0])

0.9999999999999998
